# surge analysis on IOC tide gauge
case study: cyclone FUNG WONG

## Install

Libraries needed for this notebook: 

`pip install searvey hvplot utide ipykernel geoviews pyarrow copernicusmarine`

In [ ]:
import searvey
import utide
import pandas as pd
import hvplot.pandas
import hvplot.xarray
import copernicusmarine as cm

from shapely.geometry import box

In [ ]:
ioc_stations = searvey.get_ioc_stations()
ioc_stations

define a bounding box for the area of interest

In [ ]:
lon_min, lat_min, lon_max, lat_max  = 100.0, -5.0, 130.0, 25.0
bbox = box(lon_min, lat_min, lon_max, lat_max)  # lon_min, lat_min, lon_max, lat_max

In [ ]:
selected_stations = ioc_stations[ioc_stations.within(bbox)]
selected_stations

In [ ]:
plot_ = selected_stations.hvplot.points(
    geo=True,
    tiles=True,
    hover_cols=["ioc_code"],
    s=100,
    line_color='k',
    title = "stations in the selected region"
).opts(width=600, height=700)
plot_

# overlay with copernicus data
get your credentials at https://data.marine.copernicus.eu/

Global Significant wave height 3 Hourly data 

In [ ]:
startdate = pd.Timestamp(2025, 11, 9)
enddate = pd.Timestamp(2025, 11, 12)
ds_wave = cm.open_dataset(
    dataset_id="cmems_mod_glo_wav_anfc_0.083deg_PT3H-i", 
    start_datetime=startdate, 
    end_datetime=enddate, 
    minimum_longitude = lon_min,
    maximum_longitude = lon_max,
    minimum_latitude = lat_min,
    maximum_latitude = lat_max,
    )
max_Hs = ds_wave.VHM0.max(dim="time")
max_Hs = max_Hs.compute() # to load data into RAM
max_Hs

In [ ]:
max_Hs.hvplot.image(cmap = "rainbow4", geo=True).opts(height = 800, width=800) * plot_

Global Ocean Physics Analysis and Forecast - 1 Hourly

In [ ]:
startdate = pd.Timestamp(2025, 11, 9)
enddate = pd.Timestamp(2025, 11, 12)
ds_ocean = cm.open_dataset(
    dataset_id="cmems_mod_glo_phy_anfc_0.083deg_PT1H-m", 
    start_datetime=startdate, 
    end_datetime=enddate, 
    minimum_longitude = lon_min,
    maximum_longitude = lon_max,
    minimum_latitude = lat_min,
    maximum_latitude = lat_max,
    )
max_elev = ds_ocean.zos.max(dim="time").isel(depth=0) # ! the dataset is 3D, you need to select a depth layer
max_elev = max_elev.compute() # to load data into RAM
max_elev

In [ ]:
max_elev.hvplot.image(cmap = "rainbow4", geo=True).opts(height = 800, width=800) * plot_

# detide function

In [ ]:
def surge(ts: pd.Series, lat: float, rsmp: int = None):
    ts0 = ts.copy()
    OPTS = {
        "constit": "auto",
        "method": "ols", # ols is faster and good for missing data (Ponchaut et al., 2001)
        "order_constit": "frequency",
        "Rayleigh_min": 0.97,
        "lat": lat,
        "verbose": True,
    }
    if rsmp is not None:
        ts = ts.resample(f"{rsmp}min").mean()
        ts = ts.shift(freq=f"{rsmp / 2}min")
    coef = utide.solve(ts.index, ts, **OPTS)
    tidal = utide.reconstruct(ts0.index, coef, verbose = OPTS['verbose'])
    return pd.Series(data=ts0.values - tidal.h, index=ts0.index)


# download the data

create a data folder with `raw` and `surge` subfolders

In [ ]:
! mkdir -p data/{raw,surge}

let's first download all the station in the bounding box

In [ ]:
drop_columns = ["bat"]

def serialize(d): # to export metadata
    out = {}
    for k, v in d.items():
        if v is not pd.NA and not pd.isna(v):
            out[k] = str(v)
    return out

In [ ]:
for irow, row in selected_stations.iterrows():
    lat = row.lat
    ioc_code = row.ioc_code
    if ioc_code in list(selected_stations.keys()):
        df_raw = searvey.fetch_ioc_station(
            ioc_code,
            pd.Timestamp.now()-pd.Timedelta(days=90), # we need at least 90 days (in theory we'd need more..) to remove properly tidal constituents
            pd.Timestamp.now()
        )
        df_raw.attrs = {**serialize(dict(row)), **{"signal_type": "raw"}}
        df_raw.to_parquet(f"data/raw/{ioc_code}.parquet")

In [ ]:
for irow, row in selected_stations.iterrows():
    df_raw = pd.read_parquet(f"data/raw/{row.ioc_code}.parquet")
    df_raw.loc["2025-11-01":].dropna().hvplot(
        title = f"detided signal for {row.ioc_code}, {row.location}, lat:{row.lat}, lon:{row.lon}"
    )

# detide the station

Let's look at the stations of interest

In [ ]:
detide_selection = {
    "lega" :"rad",
    "luba" :"prs",
    "quin" :"ras",
    "qing" :"rad",
    "quar" :"flt",
    "mani" :"prs",
    "subi" :"rad",
    "thsi" :"rad",
    "txil" :"rad",}

In [ ]:
for irow, row in selected_stations.iterrows():
    if row.ioc_code in list(detide_selection.keys()):
        df_raw = pd.read_parquet(f"data/raw/{row.ioc_code}.parquet")
        df_surge = surge(df_raw[detide_selection[row.ioc_code]].dropna(), lat=row.lat, rsmp=2)
        df_surge.loc["2025-11-01":].hvplot(
            title = f"detided signal for {row.ioc_code}, {row.location}, lat:{row.lat}, lon:{row.lon}"
        )
        df_surge.attrs = dict(row)
        df_surge.attrs = {**serialize(dict(row)), **{"signal_type": "detide"}}
        df_surge.to_frame().to_parquet(f"data/surge/{row.ioc_code}.parquet")

In [ ]:
(plot_*ioc_stations[ioc_stations.ioc_code.isin(list(detide_selection.keys()))].hvplot.points(
    geo=True,
    tiles=True,
    c = 'r',
    s=50,
    hover_cols=["ioc_code"],
    title = "stations that recorded a surge"
)).opts(width=800, height=800)

# look at a specific station

In [ ]:
station = "lega"
for irow, row in selected_stations.iterrows():
    if row.ioc_code == station:
        df_raw = pd.read_parquet(f"data/raw/{row.ioc_code}.parquet")
        df_raw
        station
        df_raw.loc["2025--01":].dropna().hvplot(
            title = f"signal for {row.ioc_code}, {row.location}, lat:{row.lat}, lon:{row.lon}"
        )